In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import psycopg2
import re

# settings
datapath = '/Users/mirkoknoche/MIMIC/'
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,}'.format
%matplotlib inline

# database
conn_string = open('./intern/DB_connection.txt', 'r', encoding='UTF-8').read()

# Datenbank

In [ ]:
import psycopg2
gitpath = '/Users/mirkoknoche/Google Drive/neuefische/projects/MIMIC/'
conn_string = open(gitpath + 'intern/DB_connection.txt', 'r', encoding='UTF-8').read()
conn = eval('psycopg2.connect({})'.format(conn_string.strip()))

In [ ]:
sql = """select * from vw_t_chartevents LIMIT 1000"""
df_sql = pd.read_sql(sql, conn)

In [ ]:
df_sql.head()

In [ ]:
df_sql.info()

In [ ]:
conn.close()

# inputevents_cv

Inputs are any fluids which have been administered to the patient: such as oral or tube feedings or intravenous solutions containing medications. 

Inputs exist in two separate tables: INPUTEVENTS_CV and INPUTEVENTS_MV. INPUTEVENTS_CV contains CareVue inputs, while INPUTEVENTS_MV contains Metavision inputs. Results from these tables can be unioned as observations are not duplicated across tables. For CareVue data, the rate and volume will be asynchronous, and only the CHARTTIME will be available. For rates, the CHARTTIME will correspond to a start time (when the drug was set to that rate). For volumes, the CHARTTIME will correspond to an end time.

| column | Description | Type |
|:-------|:------------|------|
| ROW_ID | ID from Database | numeric |
| SUBJECT_ID | Unique ID of a patient | numeric |
| HADM_ID | Unique ID of a patient hospital stay | numeric |
| ICUSTAY_ID | Unique ID of a patient ICU stay | numeric |
| CHARTTIME | time at which the measurement was charted at the bedside | datetime |
| ITEMID | Identifier for a single measurement type D_ITEMS.csv | numeric |
| AMOUNT | amount of a drug or substance administered to the patient either between the STARTTIME and ENDTIME | numeric |

In [ ]:
c = 1000000
df_input_cv = pd.read_csv(datapath + 'INPUTEVENTS_CV.csv', nrows=10000)

In [ ]:
sql = """select * from inputevents_cv 
        where subject_id in (select subject_id from diagnoses_icd di where ICD9_CODE = '51881')
        LIMIT 100000"""
df_input_cv = pd.read_sql(sql, conn)

In [ ]:
df_input_cv.shape

In [ ]:
df_input_cv.info()

We can see that already in the first 100000 rows some HADM_ID's are missing. Let us show how many, if we import only this column.

In [ ]:
sql = """select hadm_id from inputevents_cv """
df_test = pd.read_sql(sql, conn)

In [ ]:
df_test.shape

In [ ]:
df_test.isna().sum()

In [ ]:
print('Missing HADM_ID in percent: {:.2f}%'.format(float(df_test.isna().sum()) / df_test.shape[0] *100))

The amount of missing HADM_ID's ist tiny. We ignore these missing entrys.

In [ ]:
df_input_cv.head()

In [ ]:
sql = """select row_id from inputevents_cv """
df_test2 = pd.read_sql(sql, conn)

In [ ]:
df_test2.shape

In [ ]:
df_test2.nunique()

# Find extubations

In [14]:
conn = eval('psycopg2.connect({})'.format(conn_string.strip()))

In [ ]:
sql = """select i2.hadm_id, v.* 
        from ventsettings v
        inner join icustays i2 on i2.icustay_id = v.icustay_id 
        where (v.mechvent + v.oxygentherapy + v.extubated + v.selfextubated) > 0
            and i2.hadm_id in (select * from hadm_overview) 
        order by icustay_id, charttime """
df = pd.read_sql(sql, conn)

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
df.info()

This dataframe contains all entries from the table chartevents of the selected hadm_id's. We have to find the timestamp of the extubations and we have to check if the first extubation is succesfull.

## Getting the timestamp

Checking the exturbations

In [ ]:
df_count_ex = df.groupby('icustay_id').extubated.sum()

In [ ]:
df[(df.extubated == 1)].shape

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
ax.hist(df_count_ex)
plt.ylabel('Amount of ICU-stays')
plt.xlabel('Extubations')
plt.title('Histrogram of the extubations')
plt.savefig('./images/hist_extubations_per_icustay.png')
plt.show();

Many ICU stays have no explicit documented extubation. We need to extract the id's and find another way.

In [ ]:
# Building the difference for each row in 'mechvent' to get 
sql = """SELECT i2.hadm_id,
            v.icustay_id,
            v.charttime,
            v.mechvent,
            v.oxygentherapy,
            v.extubated,
            v.selfextubated,
            v.mechvent - lag(v.mechvent, 1) OVER (PARTITION BY v.icustay_id ORDER BY v.icustay_id, v.charttime) AS ex
          FROM ventsettings v
          JOIN icustays i2 ON i2.icustay_id::numeric = v.icustay_id
          WHERE (v.mechvent + v.oxygentherapy + v.extubated + v.selfextubated) > 0 AND (i2.hadm_id IN ( SELECT hadm_overview.hadm_id
                   FROM hadm_overview))"""
df_extub = pd.read_sql(sql, conn)

In [ ]:
df_extub.shape

In [ ]:
df_extub.head()

Now we have a new column 'ex'. If the value of 'mechvent' fall from 1 to 0, we get a -1, otherwise we get a +1. Now we can use the timestamp where ex is -1 as our extubation timestamp.

Saving that query as view: `vw_extubations`

In [ ]:
# Check the new column versus the documented extubations.
sql = """select * 
        from vw_extubations ve
        where ve.icustay_id in (select distinct icustay_id 
                                from vw_extubations ve 
                                where extubated = 1)
            and ve.extubated = 1
            and ve.ex <> -1
        order by ve.icustay_id, ve.charttime"""
df_check_ext1 = pd.read_sql(sql, conn)

In [ ]:
df_check_ext1.head(5)

In [ ]:
df_check_ext1.shape

Of 4,960 documented extubations, we cannot determine the expected flag by 1,244 entries as expected. For a better understanding we get a closer look to one icustay_id.

In [ ]:
df_extub[(df.icustay_id == 200203)].sort_values(by='charttime')

As you can see, the oxygentherapy has been documented here first and then the extubation. Due to the temporal proximity and the presumably dynamic documentation of an intensive care unit, the end of mechanical ventilation is determined as extubation. The time is needed to extract the last data before the extubation from the table chartevents. Therefore, a very precise time is not decisive here.

In [ ]:
# Creating query for extubation timestamps
sql = """select distinct i2.hadm_id, ve.icustay_id, ve.charttime
        from vw_extubations ve 
        inner JOIN icustays i2 ON i2.icustay_id::numeric = ve.icustay_id
        where ve.ex = -1
        group by i2.hadm_id, ve.icustay_id, ve.charttime
        order by ve.icustay_id"""
df_extub_time = pd.read_sql(sql, conn)

In [ ]:
df_extub_time.head()

Some icustay's includes more than one extubation. Saving that query as: `vw_timestamp_extubation`

## Create the label

In [ ]:
# Get an overview of the number of extubations
sql = """SELECT distinct i2.hadm_id, v.icustay_id, extu.ex, extu.amount,
            case 
                when extu.amount = 1 then 1
                when extu.amount is null then -1
                else 0
            end as label
        FROM ventsettings v
        inner JOIN icustays i2 ON i2.icustay_id::numeric = v.icustay_id
        left join (select distinct ve.icustay_id, ve.ex , count(ve.icustay_id) as amount
                    from vw_extubations ve 
                    where ve.ex = -1
                    group by ve.icustay_id, ve.ex) extu on extu.icustay_id = v.icustay_id 
        WHERE i2.hadm_id IN ( SELECT hadm_overview.hadm_id FROM hadm_overview)
        ORDER BY v.icustay_id"""
df_ex_ts = pd.read_sql(sql, conn)

In [ ]:
df_ex_ts.head()

In [ ]:
df_ex_ts.shape

In [ ]:
df_ex_ts.hadm_id.nunique()

In [ ]:
df_ex_ts.icustay_id.nunique()

In [ ]:
df_ex_ts.amount.fillna(0, inplace=True)

In [ ]:
df_ex_ts.groupby('label').icustay_id.count()

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
x_val = ['no extubation','succes','failed']
y_val = df_ex_ts.groupby('label').icustay_id.count()
ax.bar(x= x_val, height=y_val)

for _, value in enumerate(y_val):
    plt.text(x_val[_], value, str(value), fontsize=14,
            horizontalalignment='center', va='bottom')

plt.ylabel('Amount of ICU-stays')
plt.xlabel('Extubations')
plt.title('Counting the extubations of an ICU stay')
plt.savefig('./images/extubations_per_icustay with label.png')
plt.show();

In [ ]:
df_ex_ts.groupby('label').icustay_id.count().values

In [ ]:
x = [1614, 2069, 3883]

In [ ]:
sum(x)

In [ ]:
df.groupby('extubated').count()

In [ ]:
err_ext = list(df_count_ex[df_count_ex == 0].index)

In [ ]:
len(err_ext)

In [ ]:
sql = """select * from transfers"""
df_icuunit = pd.read_sql(sql, conn)

df_icuunit = df_icuunit[df_icuunit.icustay_id.isin(err_ext)]
df_icuunit = df_icuunit[['icustay_id', 'curr_careunit']]
df_icuunit.drop_duplicates(inplace=True)

In [ ]:
df_plot = df_icuunit.groupby('curr_careunit').count()

In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(1, 1, 1)
ax.bar(list(df_plot.index), height=list(df_plot.values.flatten()))
plt.ylabel('Missing extubations')
plt.xlabel('ICU units')
plt.title('Not documented extubations')
plt.savefig('./images/missing_extubations_per_unit.png')
plt.show();

In [ ]:
 df_plot

In [ ]:
df_qry = df[df.icustay_id == 200063]

In [ ]:
df_qry.head()

In [ ]:
df_qry['extube'] = df_qry['mechvent'].diff()
# df_qry['extube'] = df_qry['extube'].pow(2)

In [ ]:
df_qry['oxy'] = df_qry['oxygentherapy'].diff()
# df_qry['oxy'] = df_qry['oxy'].pow(2)

In [ ]:
df_qry.extube.fillna(0, inplace=True)
df_qry.oxy.fillna(0, inplace=True)

In [ ]:
df_qry.loc[160:180]

In [ ]:
df_qry[(df_qry['extube'] != 0) | (df_qry['oxy'] != 0)]

**Oxygentherapy** is ventilation without intubation

## Check extubation

In [ ]:
icustay_ids = df['icustay_id'].unique()
print(len(icustay_ids))

In [ ]:
sql = """select distinct icustay_id 
        from vw_extubations ve 
        where extubated = 1"""
df_icu_ext = pd.read_sql(sql, conn)

In [ ]:
df_icu_ext.shape

In [ ]:
print(f'Documentation of extubations: {df_icu_ext.shape[0] / len(icustay_ids) * 100:.1f} %')

There are 4,123 ICU stays with explicit documentated extubation. And 7,513 ICU stays totally.

In [ ]:
df['extube'] = df['mechvent'].diff()
df.extube.fillna(0, inplace=True)

In [ ]:
df.head()

In [ ]:
df[(df['extube'] < 0) & (~df['icustay_id'].isin(err_ext))].sample(10)

Möglicherweise ist die Beendigung der mechanischen Beatmung ein häufigeres und ebenso genaues Merkmal.We need another marker to get the timestamp of the extubation. 

In [ ]:
df[(df['extubated'] == 1)].groupby('extube').count()

In [ ]:
for i in icustay_ids:
    df[(df['icustay_id'] == i)]['extube'] = df[(df['icustay_id'] == i)].mechvent.diff()

In [ ]:
df.groupby('extube').count()

In [13]:
conn.close()

In [ ]:
df.drop('extube', axis=1, inplace=True)

# GET THE HOLY DATA-SHIT

## Patient_Group

In [15]:
sql = """select * from vw_patient_group"""
df_goal = pd.read_sql(sql, conn)

In [16]:
df_goal.shape

(5848, 13)

In [17]:
df_goal.to_csv('data/patient_group.csv', encoding='UTF-8', index=False)

## Events from chartevents

In [23]:
sql = """select * from chartev_values"""
df1 = pd.read_sql(sql, conn)

In [24]:
df1.head()

,hadm_id,icustay_id,itemid,charttime,value,valuenum,valueuom,item,storetime,cgid,warning,error,resultstatus,stopped
0,"100,016.0","217,590.0","220,045.0",2188-05-28 08:00:00,91,91.0,bpm,hr,2188-05-28 08:02:00,"15,623.0",0.0,0.0,None,None
1,"100,016.0","217,590.0","220,181.0",2188-05-28 08:00:00,88,88.0,mmHg,blood_pr,2188-05-28 08:02:00,"15,623.0",0.0,0.0,None,None
2,"100,016.0","217,590.0","220,210.0",2188-05-28 08:00:00,18,18.0,insp/min,resp_rat,2188-05-28 08:02:00,"15,623.0",0.0,0.0,None,None
3,"100,016.0","217,590.0","220,277.0",2188-05-28 08:00:00,100,100.0,%,SaO2,2188-05-28 08:02:00,"15,623.0",0.0,0.0,None,None
4,"100,016.0","217,590.0","220,339.0",2188-05-28 08:00:00,5,5.0,cmH2O,PEEP,2188-05-28 07:36:00,"20,803.0",0.0,0.0,None,None


In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74306 entries, 0 to 74305
Data columns (total 14 columns):
hadm_id         74306 non-null float64
icustay_id      74306 non-null float64
itemid          74306 non-null float64
charttime       74306 non-null datetime64[ns]
value           73903 non-null object
valuenum        73992 non-null float64
valueuom        59637 non-null object
item            74306 non-null object
storetime       74220 non-null datetime64[ns]
cgid            74220 non-null float64
warning         33854 non-null float64
error           33854 non-null float64
resultstatus    11633 non-null object
stopped         40377 non-null object
dtypes: datetime64[ns](2), float64(7), object(5)
memory usage: 7.9+ MB


In [26]:
df1.to_csv('data/chartevents_pre_extub.csv', encoding='UTF-8', index=False)

In [ ]:
df1.value.replace('None', np.nan, inplace=True)

In [ ]:
df1.value.isnull().sum()

In [ ]:
df1.value = df1.value.astype(float)

In [ ]:
df_chartev = pd.pivot_table(df1, index='icustay_id', columns='item', values='value')

In [ ]:
df_chartev.shape

### Merge into patien_group

In [ ]:
df_goal = df_goal.merge(df_chartev, how='inner', left_on='icustay_id', right_on='icustay_id')

In [ ]:
df_goal.head()

In [ ]:
df_goal.shape

## Events from labevents

In [18]:
sql = """select * from vw_labev_values"""
df2 = pd.read_sql(sql, conn)

In [20]:
df2.head(10)

,hadm_id,icustay_id,itemid,value,ts,item
0,100016,"217,590.0",50802,2,2188-05-24 17:24:00,base_excess
1,100016,"217,590.0",50820,7.43,2188-05-24 17:24:00,pH
2,100016,"217,590.0",50821,86,2188-05-24 17:24:00,Pa02
3,100016,"217,590.0",50912,1.1,2188-05-28 04:16:00,creatinin
4,100016,"217,590.0",51221,31.1,2188-05-28 04:16:00,hct
5,100016,"217,590.0",51222,10.3,2188-05-28 04:16:00,hb
6,100017,"258,320.0",50802,4,2103-03-11 05:10:00,base_excess
7,100017,"258,320.0",50820,7.52,2103-03-11 05:10:00,pH
8,100017,"258,320.0",50821,171,2103-03-11 05:10:00,Pa02
9,100017,"258,320.0",50912,0.8,2103-03-11 03:05:00,creatinin


In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35188 entries, 0 to 35187
Data columns (total 6 columns):
hadm_id       35188 non-null int64
icustay_id    35188 non-null float64
itemid        35188 non-null int64
value         35188 non-null object
ts            35188 non-null datetime64[ns]
item          35188 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.6+ MB


In [22]:
df2.to_csv('data/labevents_pre_extub.csv', encoding='UTF-8', index=False)

In [9]:
df2.value = df2.value.str.replace('^DISREGARD PREVIOUS RESULT.*', '')

In [10]:
df2.value.replace('', np.nan, inplace=True)
df2.value.replace('-', np.nan, inplace=True)

In [11]:
df2.value.unique()

array(['2', '7.43', '86', '1.1', '31.1', '10.3', '4', '7.52', '171',
       '0.8', '41.4', '14.3', '10', '7.41', '39', '40.5', '14.0', '7.39',
       '365', '1.0', '42.0', '5', '7.47', '143', '0.7', '44.8', '15.2',
       '6', '7.48', '62', '24.7', '7.6', '13', '7.49', '122', '0.5',
       '23.6', '8.1', '0', '156', '28.8', '9.8', '7.46', '105', '0.6',
       '26.7', '10.1', '-2', '7.36', '146', '26.1', '8.3', '7.45', '257',
       '27.4', '10.7', '19', '99', '1.6', '27.7', '9.3', '7.40', '402',
       '31.5', '9.7', '1', '7.44', '72', '29.9', '7.35', '127', '28.2',
       '9.6', '7.33', '163', '10.8', '3', '76', '35.5', '11.8', '83',
       '37.3', '12.3', '8', '7.42', '27.1', '9.2', '180', '0.9', '8.9',
       '95', '27.6', '9.4', '-3', '7.37', '113', '-8', '7.32', '145',
       '0.4', '29.0', '390', '27.2', '-6', '201', '11.2', '90', '1.8',
       '43.5', '14.4', '12', '34.4', '12.4', '30.9', '10.5', '112',
       '35.4', '7.17', '5.2', '27.3', '9.0', '85', '33.9', '12.1', '-9',
   

In [ ]:
df2.value = df2.value.astype(float)

In [ ]:
df_labev = pd.pivot_table(df2, index='hadm_id', columns='item', values='value')

In [ ]:
df_labev.shape

### Merge into patient_group

In [ ]:
df_goal = df_goal.merge(df_labev, how='inner', left_on='hadm_id', right_on='hadm_id')

In [ ]:
df_goal.shape

In [ ]:
df_goal.head()

In [ ]:
df_goal.icustay_id.nunique()

In [ ]:
df_goal.groupby('hadm_id').count()

In [27]:
conn.close()